In [348]:
import math
import string
import operator as ops

In [349]:
class Range:
    def __init__(self, start, stop=1, step=1):
        self.start = start
        self.stop = stop
        self.step = step
        self.value = None
        self.result_type = Array
        self.report = ['start', 'stop', 'step']
        
    def evaluate(self):
        self.value = list(range(self.start, self.stop+1, self.step))
        self.value = Array(self.value)
        return self.value

In [350]:
def tetration(n, m):
    r = n
    m = int(m)
    for i in range(m):
        r = n ** r
    return r

tetration(2, 3)

65536

In [351]:
class Expression:
    def __init__(self, a, b, op):
        self.a, self.b = a, b
        if type(a) is str:
            self.a = float(a)
        if type(b) is str:
            self.b = float(b)
        self.op = op
        self.op_list = {
            '+': ops.add,
            '*': ops.mul,
            '-': ops.sub,
            '/': ops.truediv,
            '//': ops.floordiv,
            '**': ops.pow,
            '^': ops.pow,
            '^^': tetration,
        }
        self.report = ['a', 'op', 'b']
        
    def evaluate(self):
        op_name = self.op_list[self.op]
        self.value = op_name(self.a, self.b)
        return self.value

In [352]:
class Array:
    def __init__(self, terms=[]):
        self.data = []
        for t in terms:
            if type(t) is list:
                self.data.append(array(t))
            else:
                if t not in list(',;'):
                    self.data.append(t)
    
    def apply(self, op, b, data=None):
        if not data:
            data = self.data
        result = Array()
        for d in data:
            if type(d) is Array:
                result.append(self.apply(op, b, data=d))
            elif type(d) in [int, float]:
                result.append(op(d, b))
        return result
    
    def append(self, x):
        self.data.append(x)
    
    def __add__(self, b): return self.apply(ops.add, b)
    def __mul__(self, b): return self.apply(ops.mul, b)
    def __sub__(self, b): return self.apply(ops.sub, b)
    def __truediv__(self, b): return self.apply(ops.truediv, b)
    def __floordiv__(self, b): return self.apply(ops.floordiv, b)
    def __pow__(self, b): return self.apply(ops.pow, b)
    
    def __str__(self):
        items = ', '.join(map(str, self.data))
        items = '['+items+']'
        return items

In [353]:
# a = [3,3,3]
# b = [4,4,4]
# a@b

In [354]:
class Block:
    def __init__(self, components=None, parsed=None, parser=None, r=None, block_type=None):
        self.components = components
        self.parser = parser
        self.children = []
        self.dtype = None
        self.block_type = block_type
        self.type = block_type
        self.report = []
        
        self.parsed = None
        if self.components:
            comp_vals = []
            for q in self.components:
                if type(q) in [Token]:
                    q = q.string
                    
                if type(q) in [Block]:
                    if q.like('num'):
#                         q = int(q)
                        q = q.evaluate()
                elif q.isnumeric():
                    q = int(q)
                comp_vals.append(q)
            self.parsed = parser(*r(comp_vals))
            
    def add(self, x):
        self.children.append(x)
    
    def evaluate(self):
        return self.parsed.evaluate()
    
    def print(self, level=0, label='Block'):
        indent = ' '*2
#         print(self.type)
#         print(indent + str(self) + '; ' + type(self.parsed).__name__ + '; ' + str(self.type))
        print(indent*level + label + ': ' + type(self.parsed).__name__ + '; ' + str(self.type))
        for c in self.children:
            c.print(level=level+1)
#         for k, v in vars(self.parser).items():
#             print('{}{}:{}'.format(indent, k, v))
        if self.parsed:
            for k in self.parsed.report:
                v = getattr(self.parsed, k)
                print(indent*(level)+'{}{}:{}'.format(indent, k, str(v)))
        if self.report:
            for k in self.report:
                v = getattr(self, k)
                print(indent*(level)+'{}{}:{}'.format(indent, k, str(v)))
                
    def has_tokens(self):
        return any(type(b) is Token for b in self.children)
    
    def like(self, test):
        return self.type and (self.type == test or self.type.startswith(test))
        
    def __getitem__(self, i):
        return self.children[i]
    
    def __setitem__(self, i, j):
        self.children[i] = j

In [355]:
class Token(Block):
    def __init__(self, string='', token_type=None):
        super().__init__()
        self.string = string
        self.token_type = token_type
        self.type = self.token_type
        self.report = ['string']
    
    def print(self, **kwargs):
        print(str(self))
    
    def __str__(self, **kwargs):
#         super().print(label='Token', **kwargs)
        return 'Token: '+self.string+': '+self.type

In [356]:
class Program:
    def __init__(self, source):
        self.source = source
#         Create the root node
        self.tree = Block(block_type='root')
        
#         List of characters and their corresponding type
        self.char_sets = {
            'op': '!@#$%^&*/-+<>',
            'syntax': '()[]{},;:=|',
            'letter': string.ascii_lowercase,
            'numeric': string.digits + '.-'
        }
#         List of syntactical patterns to match to generate the program's structure
#         i.e., the grammar
        self.patterns = {
#             'range': ['numeric', ':', 'numeric']
            'range1': [
                lambda x: x[0].like('num') and x[1].string == ':' and x[2].like('num') and x[3].string == ':' and x[4].like('num'),
                lambda x: [x[0], x[2], x[4]],
                Range,
                5
            ],
            'range2': [
#                 lambda x: [x[0].like('num') and x[1] == ':' and x[2].like('num')],
#                 lambda x: x[0].like('num') and x[1] == ':' and x[2].like('num'),
                lambda x: x[0].like('num') and x[1].string == ':' and x[2].like('num'),
                lambda x: [x[0], x[2]],
                Range,
                3
            ],
            'expression': [
                lambda x: x[0].like('num') and x[1].like('op') and x[2].like('num'),
                lambda x: [x[0], x[2], x[1]],
                Expression,
                3
            ]
        }
        
        self.lex()
        self.parse(self.tree)
    
    def match_types(x, y):
        return all(xi.like(y[i]) for i, xi in enumerate(x))
    
    def char_type(self, x):
#         return list(filter(lambda x: k for k, v in self.char_sets if x in v))[0]
        return [k for k, v in self.char_sets.items() if x in v][0]
    
    def lex(self):
        statements = self.source.replace('|', '\n').split('\n')
#         Remove empty lines
        statements = list(filter(None, statements))
#         Loop through lines in program
        for s in statements:
            statement_parse = Block(block_type='section')
            token = Token(s[0], token_type=self.char_type(s[0]))
            for c in s[1:]:
                c_type = self.char_type(c)
#                 If character matches the type of the current token, append it
                if c_type == token.token_type:
                    token.string += c
#                 Otherwise, store the token and start a new one
                else:
                    statement_parse.add(token)
                    token = Token(c, token_type=c_type)
            statement_parse.add(token)
            self.tree.add(statement_parse)
    
    def parse(self, block, level=0):
        for j, b in enumerate(block.children):
            self.parse(b)
        i = 0
        for k, v in self.patterns.items():
            pattern, result, block_type, num = v
#             num = 3
            section = block[i:i+num]
#             r = lambda x: x[0].like('num') and x[1] == ':' and x[2].like('num')
            if len(section) >= num:
#                 print(list(map(str, section)), pattern(section), section[1].like('num'))
                if pattern(section):
#                     print(True, list(map(str, block[i:i+num])))
#                     block[i:i+num] = Block(section, parser=block_type, r=result, block_type='numeric')
                    block[i:i+num] = [Block(section, parser=block_type, r=result, block_type='numeric')]
                    self.parse(block)
                    break
#                 use has_tokens?
                    
    def execute(self, node=None):
#         Default node is the root node
        if not node:
            node = self.tree
        
#         Loop through nodes in tree
        for b in node:
            if b.parser in [Range, Expression]:
                b.parsed.evaluate()
                print(b.parsed.value)
#             Recursively execute subnodes
            else:
#                 for v in b:
#                     self.execute()
               self.execute(node=b)
                    
    def print_tree(self):
        print('\n')
        self.tree.print()
    

In [359]:
tests = """
3:10**2
5:30
2:18**3
1:10*9
3:30:3
1+2+3
8*9+2
5+10
8*3
20/4
932//7
5+8.3
2**8
2^8
2^^3
"""

M = Program(tests)
M.execute()
# vars(M.tree[0].parsed)
M.print_tree()
# M.tree[0].components[1].like('op')
# M.tree[-1].parser#.components[1].token_type

[9, 16, 25, 36, 49, 64, 81, 100]
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[8, 27, 64, 125, 216, 343, 512, 729, 1000, 1331, 1728, 2197, 2744, 3375, 4096, 4913, 5832]
[9, 18, 27, 36, 45, 54, 63, 72, 81, 90]
[3, 6, 9, 12, 15, 18, 21, 24, 27, 30]
6
74
15
24
5.0
133
13.3
256
256
65536


Block: NoneType; root
  Block: NoneType; section
    Block: Expression; numeric
      a:[3, 4, 5, 6, 7, 8, 9, 10]
      op:**
      b:2
  Block: NoneType; section
    Block: Range; numeric
      start:5
      stop:30
      step:1
  Block: NoneType; section
    Block: Expression; numeric
      a:[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
      op:**
      b:3
  Block: NoneType; section
    Block: Expression; numeric
      a:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
      op:*
      b:9
  Block: NoneType; section
    Block: Range; numeric
      start:3
      stop:30
      step:3
  Block: NoneType; section
    Block: Expression; numeric
      a:

In [358]:
# M.__dict__
M.tree[0][1].type

IndexError: list index out of range

In [ ]:
bool([False])
f = [3, 7, 2, 1, 8]
f[0:3]